# Movie Recommendation using SentenceBERT

In [21]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

movies = pd.read_parquet("videos.parquet")
movies = movies[['item_id','video_title','author_title','video_description','category_title']].iloc[:30000]
movies.head(5)

,item_id,video_title,author_title,video_description,category_title
0,video_165654,MSI Pro MP241X недоОБЗОР (РЕШЕНИЕ ПРОБЛЕМЫ С М...,Silvi,В видео я обывательским взглядом расскажу про ...,Технологии и интернет
1,video_1173704,Наложение пястно фаланговой повязки на кисть,"УЦ ""Академия Безопасности""",Видео с канала УЦ Академия безопасности (ab-dp...,Образование
2,video_23927,SilverstoneF1 Sochi Pro и Neoline x cop 6000s ...,Artur48,SilverstoneF1 Sochi Pro и Neoline x cop 6000s ...,Авто-мото
3,video_1003780,БОЛЬНИЦА в Brookhaven! ДОКТОР ПУПКИН спас ЖЕНИ...,ПАПА ДОЧКИ Games,Играем в Роблокс (Roblox) - БОЛЬНИЦА в Brookha...,Детям
4,video_105383,"Вебинар ""Особенности трудоустройства граждан Б...","ЗАО ""Сплайн-Центр""","10.08.2023 Вебинар ""Особенности трудоустройств...",Бизнес и предпринимательство


In [22]:
movies[movies.isnull().any(axis=1)]
movies['index'] = [i for i in range(0, len(movies))]
movies = movies.dropna()

In [23]:
def get_title(index):
    return movies[movies.index == index]["video_title"].values[0]

def get_index(item_id):
    return movies[movies.item_id == item_id]["index"].values[0]

In [24]:
s = pd.read_csv('sample_submission.csv')
len(s)

97240

In [25]:
h = pd.read_csv('hack')
h


,index,date,user_id,item_id,watch_time,is_autorized,date_short,active
0,6,2023-08-21 15:21:19+03:00,user_22148320,video_1366097,32,0,2023-08-21,True
1,9,2023-08-21 00:35:41+03:00,user_14452025,video_283933,2142,0,2023-08-21,True
2,10,2023-08-21 15:30:12+03:00,user_17900299,video_1177349,4162,0,2023-08-21,True
3,21,2023-08-21 01:38:27+03:00,user_13198236,video_805119,82,0,2023-08-21,True
4,30,2023-08-21 15:20:01+03:00,user_633255,video_786707,2741,1,2023-08-21,True
...,...,...,...,...,...,...,...,...
641077,2439633,2023-08-21 01:49:45+03:00,user_529894,video_283933,3990,0,2023-08-21,True
641078,2439635,2023-08-21 19:29:55+03:00,user_6904201,video_897248,392,0,2023-08-21,True
641079,2439637,2023-08-21 11:24:13+03:00,user_5350303,video_937559,2041,0,2023-08-21,True
641080,2439638,2023-08-21 16:44:47+03:00,user_11895299,video_853811,101,0,2023-08-21,True


,index,date,user_id,item_id,watch_time,is_autorized,date_short,active
0,6,2023-08-21 15:21:19+03:00,user_22148320,video_1366097,32,0,2023-08-21,True
1,9,2023-08-21 00:35:41+03:00,user_14452025,video_283933,2142,0,2023-08-21,True
2,10,2023-08-21 15:30:12+03:00,user_17900299,video_1177349,4162,0,2023-08-21,True
3,21,2023-08-21 01:38:27+03:00,user_13198236,video_805119,82,0,2023-08-21,True
4,30,2023-08-21 15:20:01+03:00,user_633255,video_786707,2741,1,2023-08-21,True
...,...,...,...,...,...,...,...,...
641077,2439633,2023-08-21 01:49:45+03:00,user_529894,video_283933,3990,0,2023-08-21,True
641078,2439635,2023-08-21 19:29:55+03:00,user_6904201,video_897248,392,0,2023-08-21,True
641079,2439637,2023-08-21 11:24:13+03:00,user_5350303,video_937559,2041,0,2023-08-21,True
641080,2439638,2023-08-21 16:44:47+03:00,user_11895299,video_853811,101,0,2023-08-21,True


In [27]:
import numpy

bert = SentenceTransformer('bert-base-nli-mean-tokens')

#просчёт векторов
# sentence_embeddings = bert.encode(movies['video_title'].tolist())
# similarity = cosine_similarity(sentence_embeddings)
# numpy.savetxt("sim_video_title.csv", similarity, delimiter=",")
#
# sentence_embeddings = bert.encode(movies['author_title'].tolist())
# similarity = cosine_similarity(sentence_embeddings)
# numpy.savetxt("sim_author_title.csv", similarity, delimiter=",")
#
# sentence_embeddings = bert.encode(movies['video_description'].tolist())
# similarity = cosine_similarity(sentence_embeddings)
# numpy.savetxt("sim_video_description.csv", similarity, delimiter=",")
#
# sentence_embeddings = bert.encode(movies['category_title'].tolist())
# similarity = cosine_similarity(sentence_embeddings)
# numpy.savetxt("sim_category_title.csv", similarity, delimiter=",")

#загрузка просчитанных векторов
from numpy import genfromtxt
# similarity = genfromtxt('similarity.csv', delimiter=',')
sim1 = pd.read_csv('sim_video_description-003.csv')
sim2 = pd.read_csv('sim2_video_description.csv')
similarity = pd.concat([sim1, sim2])

In [28]:
ha = pd.read_parquet('videos.parquet')
item_id = ha[['item_id']].iloc[:97240]


In [29]:
len(item_id)

97240

In [30]:


new = pd.DataFrame()

for item in item_id:
    try:
        recommendations = sorted(list(enumerate(similarity[get_index(item_id)])), key = lambda x:x[1], reverse = True)
        print(recommendations)
        for t in recommendations:
            print(get_title(t[0]))
    except Exception as e:
        print(e)


Operands are not aligned. Do `left, right = left.align(right, axis=1, copy=False)` before operating.


In [34]:
movies = pd.read_csv("train")
movies = movies[['item_id']].iloc[:97240]

In [ ]:
for i in range(len(movies)):
    movies[i].strip('"')

In [48]:
movies.to_csv('moveee.csv', index=False)

In [36]:
s = pd.read_csv('sample_submission.csv')

In [37]:
s = s[['user_id']]

In [46]:
s.to_csv('asd.csv')

In [69]:
similarity = pd.DataFrame(s, columns=['user_id','recs'])

In [67]:
movies.to_excel('BBB.xlsx')

In [62]:
similarity.loc['recs'] = movies.item_id

In [64]:
similarity.to_csv('sad.csv')

In [70]:
similarity

,user_id,recs
0,user_26511551,NaN
1,user_29194819,NaN
2,user_29734049,NaN
3,user_955460,NaN
4,user_7065521,NaN
...,...,...
97235,user_29281681,NaN
97236,user_3912848,NaN
97237,user_28389099,NaN
97238,user_18951296,NaN


In [72]:
similarity.to_excel('AAA.xlsx',index=False)

In [41]:
similarity.loc[0] = s

In [42]:
similarity.loc[1] = movies

In [43]:
similarity

,user_id,item_id
0,user_26511551,NaN
1,NaN,['video_856096']
2,user_29734049,NaN
3,user_955460,NaN
4,user_7065521,NaN
...,...,...
97235,NaN,['video_1721228']
97236,NaN,"['video_1105684', 'video_1558559']"
97237,NaN,"['video_1738263', 'video_1738263', 'video_1738..."
97238,NaN,"['video_1569589', 'video_736731', 'video_16248..."
